<a href="https://colab.research.google.com/github/mmistroni/OpenBB_Experiments/blob/main/OpenBBFinancialAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the OpenBB Platform in Google Colab

This notebook will install the OpenBB Platform, fetch some data and prepare it for display as a bar chart.

Sign up for a free account here: https://my.openbb.co

In [ ]:
# Install the OpenBB Platform with all available extensions.
# Messages indicating package version conflicts at the end of installation can be safely ignored.

!pip install openbb
!pip install pandas_ta
!pip install openbb-charting
# There is also a nightly distribution available, openbb-nightly

In [ ]:
# Before running this cell, restart the runtime by selecting, "Restart runtime", from the "Runtime" menu.

# Import statements - for many scenarios, the only import needed will be `from openbb import obb`
from typing import Literal
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets
import pandas as pd
import pandas_ta as ta
from datetime import datetime , date
from plotly import graph_objects as go

from openbb import obb

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

def get_openbb_keys():
  with open('gdrive/My Drive/passwords/openbb.keys') as f:
    return f.readlines()[0]

def get_benzinga_keys():
  with open('gdrive/My Drive/passwords/benzinga.keys') as f:
    return f.readlines()[0]

def get_intrinio_keys():
  with open('gdrive/My Drive/passwords/intrinio.keys') as f:
    return f.readlines()[0]

def getfmpkeys():
  with open('gdrive/My Drive/passwords/fmprep.keys') as f:
    return f.readlines()[0]
def get_nasdaq_keys():
  with open('gdrive/My Drive/passwords/nasdaq.keys') as f:
    return f.readlines()[0]


In [ ]:
# Login to OpenBB Hub to retrieve stored API keys.
# https://my.openbb.co/app/platform/pat
# https://my.openbb.co/app/platform/api-keys
from google.colab import userdata

obb.account.login(pat=userdata.get('PAT_KEY'))

In [ ]:
obb.user.credentials.benzinga_api_key=get_benzinga_keys()
obb.user.credentials.intrinio_api_key=get_intrinio_keys()
#obb.user.credentials.nasdaq_api_key=get_nasdaq_keys()


In [ ]:
# Verify that the credentials from Hub were loaded successfully.


obb.user.preferences.output_type = "dataframe"
obb.user.credentials

In [ ]:
df = obb.equity.price.historical(symbol = 'SEZL', provider='fmp')[::-1]
df
#df[df['change_percent'] > 0.09]

In [ ]:
df = obb.economy.available_indicators(provider='econdb')
df[df.country.isin(['United States'])].description.values.tolist()

### Defining Tickers for Analysis


In [ ]:
# Set the output preference, if desired. The examples below use Pandas DataFrames.
from datetime import timedelta
from dateutil.relativedelta import relativedelta
TICKER = 'DIS'
TICKER2 = '^GSPC' # S&P500
SECTOR_TICKER = 'XLY'
INDUSTRY_TICKER = 'XME'
startDate = date.today() - relativedelta(years=5)

In [ ]:
from openbb_charting.charts.price_historical import price_historical
from openbb_charting.charts.generic_charts import line_chart
aapl = obb.equity.price.historical(symbol = 'AAPL', provider='fmp')
res = line_chart(aapl)
res.show()

### Return Comparison

In [ ]:
from openbb_charting.charts.generic_charts import line_chart

spy = obb.equity.price.historical(symbol = TICKER2, provider='fmp')[['close']].rename(columns={'close' : TICKER2})
dis = obb.equity.price.historical(symbol = TICKER, provider='fmp')[['close']].rename(columns={'close' : TICKER})
sector = obb.equity.price.historical(symbol = SECTOR_TICKER, provider='fmp')[['close']].rename(columns={'close' : SECTOR_TICKER})
industry = obb.equity.price.historical(symbol = INDUSTRY_TICKER, provider='fmp')[['close']].rename(columns={'close' : INDUSTRY_TICKER})

one = pd.merge(spy, dis, left_index=True, right_index=True)
two = pd.merge(one, sector,left_index=True, right_index=True)
result = pd.merge(two, industry,left_index=True, right_index=True)
histpriceconcat = result/result.iloc[0]


for i, col in enumerate([TICKER2, TICKER, SECTOR_TICKER, INDUSTRY_TICKER]):
    histpriceconcat[col].plot()
'''
plt.title('Price Evolution Comparison')
plt.xticks(rotation=70)

legendCols = [TICKER2, TICKER, f'{SECTOR_TICKER}(Sector)', f'{INDUSTRY_TICKER}(Industry)']

plt.legend(legendCols)
#plt.savefig('foo1.png', bbox_inches='tight')
#
#plt.show()

res = line_chart(histpriceconcat, title='Price evolution')
res.show()
'''

### Overview

In [ ]:
line_chart([spy, dis, sector, industry], xtitle='Price evolution',ytitle='Test' , normalize=True, same_axis=True).show()

### Overview of the company

In [ ]:
obb.equity.fundamental.overview(symbol=TICKER)

### Stock Peers

In [ ]:
import os
import requests

def get_peers(ticker):
    apiKey = getfmpkeys()
    baseUrl = f'https://financialmodelingprep.com/api/v4/stock_peers?symbol={ticker}&apikey={apiKey}'
    return requests.get(baseUrl).json()[0].get('peersList', [])

full_tickers = obb.equity.compare.peers(symbol=TICKER)[1].values.tolist()[0]
full_tickers

### Fetching some ratios


### Ratios Overview

In [ ]:
liq_ratios = ['current_ratio', 'quick_ratio', 'operating_cash_flow_per_share',  'days_of_sales_outstanding' ]
solvency_ratios = ['interest_coverage', 'debt_ratio',  'debt_equity_ratio'] #missing shareholder equity ratio
profitability_ratios = ['gross_profit_margin', 'operating_profit_margin',
                        #revenuePerShare
                        'pretax_profit_margin',
                        'net_profit_margin',
                        #Cash flow margin
                        'return_on_assets',
                        'return_on_equity',
                        'return_on_capital_employed',
                         #'price_to_sales_ratio'
                         ]

### Quarterly

In [ ]:
annual_ratios = obb.equity.fundamental.ratios(symbol=TICKER, period="annual", limit=5)
quarterly_ratios = obb.equity.fundamental.ratios(symbol=TICKER, period="quarter", limit=12)
quarterly_ratios['key'] = quarterly_ratios.apply(lambda row: f"{row['fiscal_period']}_{row['fiscal_year']}", axis=1)
# Income statement. we need this for trajectory of fundamentals
annual_income = obb.equity.fundamental.income(symbol=TICKER, period="annual", limit=5)
quarterly_income = obb.equity.fundamental.income(symbol=TICKER, period="quarter", limit=12)
quarterly_income['key'] = quarterly_ratios.apply(lambda row: f"{row['fiscal_period']}_{row['fiscal_year']}", axis=1)



### Annually

In [ ]:
annual_liquidity = annual_ratios[liq_ratios + ['fiscal_year'] ].set_index('fiscal_year')
annual_solvency = annual_ratios[solvency_ratios + ['fiscal_year'] ].set_index('fiscal_year')
annual_profitability = annual_ratios[profitability_ratios + ['fiscal_year'] ].set_index('fiscal_year')
quarterly_liquidity = quarterly_ratios[liq_ratios + ['key'] ].set_index('key')
quarterly_solvency = quarterly_ratios[solvency_ratios + ['key'] ].set_index('key')
quarterly_profitability = quarterly_ratios[profitability_ratios + ['key'] ].set_index('key')

annual_income = annual_income.set_index('fiscal_year')
quarterly_income = quarterly_income.set_index('key')


In [ ]:
annual_income.columns

### Charting.....

In [ ]:
#annual_liquidity
import matplotlib.pyplot as plt

#define subplot layout
fig, axes = plt.subplots(nrows=1, ncols=2)
annual_liquidity.apply(pd.to_numeric).plot.bar(figsize=(10,8), title='Liquidity Ratios annual', ax=axes[0])
quarterly_liquidity.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Liquidity Ratios qtr', ax=axes[1])

In [ ]:
from openbb_charting.charts.generic_charts import bar_chart

bar_chart([annual_liquidity], x='fiscal_year', y=liq_ratios, title='Liquidity Ratios annual', normalize=True).show()

### Profitability

In [ ]:
#annual_liquidity
import matplotlib.pyplot as plt

#define subplot layout
fig, axes = plt.subplots(nrows=2, ncols=1)
annual_profitability.apply(pd.to_numeric).plot.bar(figsize=(20, 10),title='Profitability Ratios annual', ax=axes[0])
quarterly_profitability.apply(pd.to_numeric).plot.bar(figsize=(20,10), title='Profitability Ratios qtr', ax=axes[1])

### Solvency

In [ ]:
import matplotlib.pyplot as plt

#define subplot layout
fig, axes = plt.subplots(nrows=1, ncols=2)
annual_solvency.apply(pd.to_numeric).plot.bar(figsize=(10,8), title='Solvency Ratios annual', ax=axes[0])
quarterly_solvency.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Solvency Ratios qtr', ax=axes[1])

### Now we will resort to the old analysis with Management, Profitability etc that we were used to do in colab

## Now Using other measures before checking financial statements in details

* Gross Profit Margin: Indicates how efficiently a company produces goods or

services by measuring the difference between revenue and the cost of goods sold (COGS).
* Net Profit Margin: Measures a company’s overall profitability by calculating the percentage of net income relative to revenue.
* Return on Assets (ROA): Assesses a company’s ability to generate profits from its assets, calculated by dividing net income by total assets.
* Return on Equity (ROE): Evaluates a company’s ability to generate returns for its shareholders by dividing net income by shareholders’ equity.
* Operating Profit Margin: Shows a company’s operating efficiency by measuring operating income as a percentage of revenue.



## TODO. follow https://colab.research.google.com/drive/1ZdGLqEfAXfiseahjIwxlRXgQbPg3-5Ol#scrollTo=LArvBJ9Qs9a2
COMPARE Across years

- check asset value (total assets, total liabilities, outstading shares, NAV
- compar across years priceearningration, priceearningtogrowthratio, earningyield
- net income, operating cf, capital expenditure, free cf, cashand cash equiv, year of cash


We are looking for htis

<ul>
    <li><b>Staying power</b>: You want to make sure a company has the financial resources to endure a downturn and come out on the other side.<li>
    <li><b>The trajectory of the fundamentals</b>: Because a stock price, over time, is connected to a company’s revenue and earnings, fundamental analysts try to see improving trends in the company’s revenue and profit. Trend spotting will be discussed at more length in Chapter 17.</li>
    <li><b>Evidence of skilled management</b>: A management team with experience navigating through the ups and downs of a business may give the fundamental analyst more confidence in the company’s future. Skilled managers can protect their company’s business from competition with strong brands, service, or quality.</li>
    <li><b>Valuation</b>: Even if a company is performing poorly, the stock can still be a good investment if the bad news is already reflected in the price. Fundamental analysts spend a great deal of their time comparing a company’s current stock price to its real value, based on what they think it’s worth. You discovered how to use the price-to-earnings ratio, or P-E, as a valuation tool in Chapter 8. In this chapter you’ll find out about the earnings yield. Fundamental analysts also look for good values using the discounted cash-flow model, explored in Chapter 11.</li>
    <li><b>Dividend payments</b>: These seemingly small cash payments can quickly add up and become a significant consideration for a fundamental analyst. Fundamental analysts also use these dividends as a way to measure the attractiveness of some stocks, as will be discussed at the end of this chapter.</li>
    </ul>
In addition, we might want to look at this. (Check what we already fetch from our superperformers loader)

<ol>
    <li>Financial Health:
        <ol>
<li>Revenue Growth Rate: Ideally, a company should have a positive revenue growth rate of at least 5% per year.</li>
<li>Profit Margin: A healthy profit margin varies by industry, but a company with a profit margin of at least 10% is generally considered financially healthy.</li>
<li>Debt-to-Equity Ratio: A ratio of less than 1 indicates that a company is financed more by equity than debt, which is generally considered financially healthy.</li>
<li>Free Cash Flow: A positive free cash flow indicates that a company is generating cash from its operations and is able to invest in growth opportunities.</li>
        </ol>
    </li>
 <li>Competitive Advantage:
    <ol>
<li>Market Share: A company with a market share of at least 20% is generally considered to have a strong competitive advantage in its industry.</li>
<li>Return on Equity: A healthy return on equity varies by industry, but a company with a return on equity of at least 15% is generally considered to have a strong competitive advantage.</li>
<li>Price-to-Earnings Ratio: A company with a price-to-earnings ratio that is lower than its peers may indicate that the company is undervalued and has a strong competitive advantage.</li>
<li>Research and Development (R&D) Spending: A company that invests at least 5% of its revenue in R&D is generally considered to be investing in innovation and maintaining its competitive advantage.</li>
     </ol>
  </li>
  <li>Management Quality:
    <ol>
<li>CEO Tenure: Ideally, a CEO should have been leading the company for at least 3–5 years to demonstrate a track record of success.</li>
<li>Executive Compensation: Executive compensation varies by industry, but excessive compensation may indicate poor management quality.</li>
<li>Employee Turnover Rate: A low employee turnover rate indicates that employees are satisfied and that the management team is effective at retaining talent.</li>
<li>Return on Assets: A healthy return on assets varies by industry, but a company with a return on assets of at least 5% is generally considered to have effective management.  </li>
      </ol>
    </li>
</ol>
    
    

### Fundamental Measures

In [ ]:
skilled_management_measures = ['return_on_assets', 'return_on_equity', 'return_on_capital_employed']
staying_power_measures = idxs = ['return_on_capital_employed', 'current_ratio', 'debt_equity_ratio', 'interest_coverage', 'cash_per_share']
valuation_measures = ['price_earnings_to_growth_ratio', 'price_earnings_ratio']
profitability_measures = ["date", "revenue", "costOfRevenues", "operatingExpenses", # will need to probably get them from net income
              "otherExpenses", "netIncome", "ebitda", "interestExpense", "incomeTaxExpense",
              "operatingIncome"]
financial_rations_measures = ['return_on_equity', 'return_on_capital_employed',"receivablesTurnover",
              "inventoryTurnover", "payablesTurnover", "debt_equity_ratio",
              "current_ratio", "quick_ratio", "cash_ratio", "interest_coverage",
              "priceToBookRatio", "dividendYield", "dividendPayoutRatio", "priceEarningsRatio",
              "priceEarningsToGrowthRatio", "cashPerShare"]

trajectory_of_fundamentals = ['revenue', 'diluted_earnings_per_share', 'consolidated_net_income']
cashflow_measures = ['symbol', 'netIncome', "operatingCashFlow", "capitalExpenditure", "freeCashFlow", "ebtida"]
trend_measures = ['operatingIncome', 'revenues', 'eps', 'dividend']

# Find net asset value

In [ ]:
ratios = obb.equity.fundamental.ratios(symbol=TICKER, period="annual", limit=5)


### Staying Power (annual)

In [ ]:
fig, axes = plt.subplots(2, 2)
print(axes)
staying_power_measures = ['fiscal_year'] + ['return_on_capital_employed', 'current_ratio', 'debt_equity_ratio', 'interest_coverage', 'cash_per_share']
skilled_mgmt_measures = ['fiscal_year'] + skilled_management_measures
valuation_measures = ['fiscal_year'] + valuation_measures
staying_power = annual_ratios[staying_power_measures].set_index('fiscal_year')
skilled_mgmt = annual_ratios[skilled_mgmt_measures].set_index('fiscal_year')
valuation = annual_ratios[valuation_measures].set_index('fiscal_year')
traj_fundamentals = annual_income[trajectory_of_fundamentals]
staying_power.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Staying power', ax=axes[0,0])
skilled_mgmt.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Skilled Mgmt', ax=axes[0,1])
valuation.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Valuation', ax=axes[1,0])
traj_fundamentals.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Traj of Fundamentals', ax=axes[1,1])

### Quarterly

In [ ]:
fig_qtr, qtr_axes = plt.subplots(2, 2)
print(axes)

staying_power_qtr = quarterly_ratios[staying_power_measures].set_index('fiscal_year')
skilled_mgmt_qtr = quarterly_ratios[skilled_mgmt_measures].set_index('fiscal_year')
valuation_qtr = quarterly_ratios[valuation_measures].set_index('fiscal_year')
traj_fundamentals_qtr = quarterly_income[trajectory_of_fundamentals]
staying_power_qtr.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Staying power', ax=qtr_axes[0,0])
skilled_mgmt_qtr.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Skilled Mgmt', ax=qtr_axes[0,1])
valuation_qtr.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Valuation', ax=qtr_axes[1,0])
traj_fundamentals_qtr.apply(pd.to_numeric).plot.bar(figsize=(20,8), title='Traj of Fundamentals', ax=qtr_axes[1,1])

###Dividends Over Time

In [ ]:
df = obb.equity.fundamental.dividends(TICKER)
df['cob'] = pd.to_datetime(df.ex_dividend_date)
df = df.set_index(df.cob)
df[df.index > startDate.strftime('%Y-%m-%d')].plot(x='cob')


### Estimates

In [ ]:
obb.equity.estimates.consensus(symbol=TICKER)

### Discounted cashflow

In [ ]:
ov_df = obb.equity.fundamental.overview(symbol=TICKER)
ov_df[ov_df[0].isin(['dcf', 'dcf_diff'])]

### Splits

In [ ]:
obb.equity.fundamental.historical_splits(TICKER)

### Financial Statement Growth

In [ ]:
obb.equity.fundamental.balance_growth(symbol=TICKER, limit=10)

In [ ]:
obb.equity.fundamental.income_growth(symbol=TICKER, limit=10)

In [ ]:
obb.equity.fundamental.cash_growth(symbol=TICKER, limit=10)

## Comparison With Stock in same sectors

Analyzing a competitor's financial health involves looking at various aspects, and there's no single "best" set of ratios. However, a good approach is to consider ratios from different categories to get a well-rounded picture. Here are some key categories and specific ratios to consider:

Profitability Ratios:

Gross Profit Margin: Measures the percentage of revenue remaining after accounting for the cost of goods sold. Indicates how efficiently a company converts sales into profit.
Operating Margin: Similar to gross margin, but factors in operating expenses, providing a clearer picture of profitability from core operations.
Return on Assets (ROA): Measures how effectively a company uses its assets to generate profit.
Return on Equity (ROE): Shows the return on investment for shareholders.
Liquidity Ratios:

Current Ratio: Assesses a company's ability to meet short-term obligations using current assets.
Quick Ratio (Acid-Test Ratio): Similar to the current ratio, but excludes inventory (considered less liquid) from current assets for a more stringent assessment.
Solvency Ratios:

Debt-to-Equity Ratio: Indicates the proportion of debt used to finance the company compared to shareholder equity. Helps assess the risk of financial distress.
Efficiency Ratios:

Inventory Turnover Ratio: Measures how often a company sells and replaces its inventory stock.
Receivables Turnover Ratio: Indicates how efficiently a company collects payments from customers.
Additional Considerations:

Industry Benchmarks: Comparing your competitor's ratios against industry averages can reveal strengths and weaknesses relative to peers.
Trends Over Time: Analyzing trends in these ratios over several periods provides valuable insights into the company's financial trajectory.
Combine Ratio Analysis with Other Information: Don't solely rely on ratios, consider news, market trends, and the company's business model for a comprehensive understanding.

### Historical Market Cap Last 5 yrs

In [ ]:
def get_marketcap_for_ticker(ticker):
  end_date = date.today()
  start_date = date(end_date.year-5, 1, 1)
  fmp_url =  f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{ticker}?from={start_date.strftime('%Y-%m-%d')}&to={end_date.strftime('%Y-%m-%d')}&apikey={getfmpkeys()}"
  data = requests.get(fmp_url).json()

  df = pd.DataFrame(data)[['date', 'marketCap']]
  df['date'] = pd.to_datetime(df.date)
  df['marketCap'] = df.marketCap / 1000000

  df = df.set_index('date')
  return df.rename(columns={'marketCap' : ticker})

dfs = [get_marketcap_for_ticker(t) for t in full_tickers]

dis = get_marketcap_for_ticker(TICKER)

from functools import reduce

alldata = reduce(lambda df, acc: pd.merge(df, acc, left_index=True, right_index=True), dfs, dis)
alldata.plot(figsize=(20, 8))



### Net Profit Margin

In [ ]:
#full_tickers
ratios = ['net_profit_margin',
          'operating_profit_margin',
          'return_on_assets',
          'current_ratio',
          'quick_ratio']

def get_measure_from_ratios(ticker, ratio_id):
  data = obb.equity.fundamental.ratios(symbol=ticker, period="annual", limit=5)[['fiscal_year', ratio_id]]
  data = data.rename(columns={ratio_id : ticker})
  return data.set_index('fiscal_year')

for ratio in ratios:
  eqt = get_measure_from_ratios(TICKER, ratio)
  dfs = [get_measure_from_ratios(t, ratio) for t in full_tickers]
  alldata = reduce(lambda df, acc: pd.merge(df, acc, left_index=True, right_index=True), dfs, eqt)[::-1]
  alldata.plot.bar(figsize=(20, 8), title = ratio.upper())
# TODO, PUT IT IN A GRID



In [ ]:
obb.equity.fundamental.ratios(symbol='AAPL', period="annual", limit=1).T

### EPS
###Adjusted EBITDA increased by 16%, and diluted adjusted EPS grew 25%
### cASH
### Inventory


<ul>
  <li>Profitability:
    <ul>
      <li>Net profit margin: Net income / Net sales (Shows how much profit each dollar of sales generates)</li>
      <li>Return on equity (ROE): Net income / Shareholder's equity (Measures profitability relative to shareholder investment)</li>
   </ul>
  </li>

  <li>Liquidity:
    <ul>
      <li>Current ratio: Current assets / Current liabilities (Indicates short-term debt repayment ability)</li>
      <li>Quick ratio: (Current assets - Inventories) / Current liabilities (More stringent measure of liquidity)</li>
    </ul>
  </li>
  <li>Efficiency:
    <ul>
      <li>Inventory turnover: Cost of goods sold / Average inventory (Measures how quickly inventory is sold)</li>
      <li>Asset turnover: Net sales / Average total assets (Indicates how effectively assets are used to generate sales)</li>
      </li>
    </ul>
  </li>
  <li>Debt:
    <ul>
      <li>Debt-to-equity ratio: Total debt / Total equity (Measures financial leverage and risk)</li>
      <li>Interest coverage ratio: EBIT / Interest expense (Indicates ability to meet interest payments)</li>
    </ul>
  </li>
  <li>Market Valuation:
    <ul>
      <li>Price-to-earnings ratio (P/E ratio): Share price / Earnings per share (Measures price relative to profitability)</li>
      <li>Enterprise value / EBITDA ratio: Enterprise value / EBITDA (Considers debt and compares cash flow potential)</li>
    </ul>
  </li>
</ul>

### Insider Tradings

In [ ]:
obb.equity.ownership.insider_trading(symbol=TICKER, limit=500)

In [ ]:
obb.equity.discovery.filings(limit=100, form_type='13F')

### OBV

In [ ]:
from openbb import obb
stock_data = obb.equity.price.historical(symbol=TICKER, start_date="2023-10-01", provider="fmp")[['close', 'volume']]
stock_data
obb.technical.obv(data=stock_data, offset=0).plot()

### Ownership

In [ ]:
import requests
import re
df  =obb.equity.fundamental.overview(symbol=TICKER)
ticker_exchange = df[df[0].isin(['exchange_short_name'])][1].values[0]
base_url = f'https://www.marketbeat.com/stocks/{ticker_exchange}/{TICKER.upper()}/institutional-ownership/'
print(base_url)
str1 = requests.get(base_url).text
string_pattern = r"Institutional Ownership Percentage.*[\d]+\.[\d]+%<\/div>"
# compile string pattern to re.Pattern object
regex_pattern = re.compile(string_pattern)
res = regex_pattern.findall(str1)[0]
float(res[res.find('strong>') + 7: res.rfind('%')])


In [ ]:
obb.equity.ownership.institutional(symbol="AAPL", provider='yfinance')

### Equity discovery

### Most Active

In [ ]:
obb.equity.discovery.active(sort="desc")

### Gainers

In [ ]:
obb.equity.discovery.gainers(sort="desc")

### Undervalued large caps

In [ ]:
obb.equity.discovery.undervalued_large_caps(sort="desc")

In [ ]:
obb.equity.discovery.top_retail(limit=5)

In [ ]:
obb.equity.discovery.undervalued_growth(sort="desc")